# 手写实现2D卷积

In [1]:
import numpy as np
import mxnet as mx
from mxnet import gluon,nd,autograd
from mxnet.gluon import nn

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#互相关运算函数，输入为X，卷积和为K
def corr2d(X,K):
    h,w = K.shape
    Y = nd.zeros((X.shape[0]-h+1,X.shape[1]-w+1),ctx=mx.gpu()) 
    #下面进行卷积运算
    for  i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K[:,:]).sum()
    return Y

In [3]:
#执行卷积计算
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]],ctx=mx.gpu())
K = nd.array([[0, 1], [2, 3]],ctx= mx.gpu())
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @gpu(0)>

## 手动实现一个二维卷积层 Conv2D，其中通过卷积操作之后，加上偏差，然后经过激活函数后输出

In [12]:
class Conv2D(nn.Block):
    def __init__(self,kernel_size,**kwargs):
        super(Conv2D,self).__init__(**kwargs)
        self.weight = self.params.get('weight',shape = kernel_size)
        self.bias = self.params.get('bias',shape=(1,))
        
    def forward(self,x):
        x = corr2d(x,self.weight.data())
        return nd.relu(x+self.bias.data())

In [13]:
conv1 = Conv2D(kernel_size=(3,3))

In [14]:
conv1.initialize(ctx=mx.gpu())

In [22]:
Y=conv1(X)

## 构造一个简单的例子来检测图像的边缘

In [16]:
X = nd.ones(shape=(6,8),ctx = mx.gpu())
X[:,2:6]=0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @gpu(0)>

In [19]:
K = nd.array([[1,-1]],ctx = mx.gpu())

In [23]:
Y=corr2d(X,K)

## 通过数据来学习核数组
## 使用gluon接口中的卷积来实现

In [21]:
conv2d = nn.Conv2D(1,kernel_size=(1,2))
conv2d.initialize(ctx=mx.gpu())

# 卷积层采用四维输入 （样本数，通道数，宽，高）

In [24]:
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

In [25]:
for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat-Y)**2
    l.backward()
    
    #梯度下降
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 4.998
batch 4, loss 0.855
batch 6, loss 0.150
batch 8, loss 0.028
batch 10, loss 0.006


In [27]:
conv2d.weight.data()


[[[[ 0.99299324 -0.9839153 ]]]]
<NDArray 1x1x1x2 @gpu(0)>